In [21]:
import torch
from get_vector import *
import numpy as np
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

In [45]:
class RNNTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, tagset_size, batch_size):
        super(RNNTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        
        self.rnn = nn.RNNCell(embedding_dim, hidden_dim)
        
        self.hidden2tag = nn.Linear(hidden_dim*10, tagset_size)
        self.hidden = self.init_hidden()
        
    
    def init_hidden(self):
        return Variable(torch.randn(self.batch_size, self.hidden_dim))


    def forward(self, vocab_size, input):
        output = []
        for i in range(vocab_size):
            self.hidden = self.rnn(input[i], self.hidden)
            output.append(self.hidden)
        rnn_output = torch.stack(output)
        tag_space = self.hidden2tag(rnn_output.view(1, -1))
        tag_scores = F.log_softmax(tag_space)
        return tag_scores

In [53]:
model = RNNTagger(10, 20, 3, 1)
loss_function = nn.NLLLoss()
inputs = Variable(torch.randn(3, 10, 10))
targets = Variable(torch.LongTensor([[0, 1, 0],[0, 0, 1],[1, 0, 0]]))
for epoch in range(10):
    for i in range(3):
        model.zero_grad()
        model.hidden = model.init_hidden()
        tag_scores = model(10, inputs[i])
        print tag_scores.size(), targets[i].size()
        loss = loss_function(tag_scores, targets[i])
        loss.backward()
        optimizer.step()

torch.Size([1, 3]) torch.Size([3])


RuntimeError: Assertion `THIndexTensor_(size)(target, 0) == batch_size' failed.  at /pytorch/torch/lib/THNN/generic/ClassNLLCriterion.c:54